In [1]:
# Importing needed library
import pandas as pd
import numpy as np
import beta_glu
import pandascharm as pc
from sklearn.metrics import r2_score
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
from Bio import SeqIO
from Bio.SeqUtils.ProtParam import ProteinAnalysis
import blosum as bl
from Bio.SubsMat.MatrixInfo import blosum62 as blosum
from Bio import AlignIO
from Bio import SeqIO
from Bio.Align.Applications import MuscleCommandline
from Bio.Align import AlignInfo
from scipy.fft import fft, ifft
import matplotlib.pyplot as plt

from sklearn.preprocessing import scale 
from sklearn import model_selection
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import train_test_split
from sklearn.cross_decomposition import PLSRegression
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.metrics import r2_score
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Lasso
from pyaaisc import Aaindex
from Bio import AlignIO


/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/Bio/SubsMat/__init__.py:126: BiopythonDeprecationWarning: Bio.SubsMat has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.substitution_matrices as a replacement, and contact the Biopython developers if you still need the Bio.SubsMat module.
  warnings.warn(


In [2]:
df = pd.read_excel('../beta-glucosidase/data/KcatKm w_ Temperatures.xlsx')

In [3]:
output = 'pNP-Glc kcat/Km (1/smM)'
encoding = 'quadrogram'
aln = '../beta-glucosidase/data/kcatKm w_Temperatures.aln'

In [4]:
X,y,holder = beta_glu.encode_temp(encoding, output, df, aln, key = None)

In [5]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.20, random_state=101)

In [6]:
all_result = []

In [7]:
# Random Forest Tuning
def RF(X_train, y_train, X_val, jack = False):
    knife = len(X_train)-1
    
    param_grid = {'max_depth': [70], 'min_samples_leaf': [1], 'min_samples_split': [2], 'n_estimators': [50]}
    rf = RandomForestRegressor()
    if jack == False:
        grid_search=GridSearchCV(rf, param_grid, cv=5, scoring='neg_mean_squared_error', return_train_score=True)
    else:
        grid_search=GridSearchCV(rf, param_grid, cv=knife, scoring='neg_mean_squared_error', return_train_score=True)
    grid_search.fit(X_train,np.ravel(y_train))
    RF_pred = grid_search.predict(X_val)
    pred_training= grid_search.predict(X_train)
    feature_imp = grid_search.best_estimator_.feature_importances_
    return RF_pred, pred_training, feature_imp

In [8]:
RF_pred , RF_train, feature_imp = RF(X_train, y_train, X_val)

In [9]:
np.where(feature_imp == np.max(feature_imp))

(array([59059]),)

In [10]:
feature_imp[6535]

0.0

In [159]:
temp_X.columns[6535]

'VRS'

In [160]:
np.where(temp_X.columns == "ENG")

(array([261]),)

In [161]:
# Feature Column
lst = ['E','G','L','Y','T','H','R','A','C','D','P','I','F','N','K','S','V','M','W','Q']
all_dct = {}
key = []
for i in lst:
    for j in lst:
        for k in lst:
            st = i+j+k
            all_dct[st] = []
all_dct['temp'] = []

In [162]:
for i, j in zip(all_dct.keys(),feature_imp):
    all_dct[i] = j
Importance_df = pd.DataFrame.from_dict([all_dct])

In [163]:
Importance_df

,EEE,EEG,EEL,EEY,EET,EEH,EER,EEA,EEC,EED,...,QQI,QQF,QQN,QQK,QQS,QQV,QQM,QQW,QQQ,temp
0,3.530718e-10,8.338272e-07,1.297487e-09,0.000005,0.0,6.400689e-12,0.000336,0.0,0.0,8.402851e-10,...,0.0,0.0,0.0,0.0,0.000003,1.034097e-07,0.0,0.0,0.0,0.345473


In [164]:
Importance_df.to_excel("RF_Feature_importance.xlsx")  

In [10]:
# XGBR tuning
def XGBR(X_train, y_train, X_val, jack = False):
    knife = len(X_train)-1
    
    param_grid = {'colsample_bytree': [0.6], 'gamma': [1.5], 'max_depth': [5], 'min_child_weight': [1], 'subsample': [0.6]}
    
    if jack == False:
        grid_search=GridSearchCV(xgb.XGBRegressor(), param_grid, cv=5, scoring='neg_mean_squared_error', return_train_score=True)
    else:
        grid_search=GridSearchCV(xgb.XGBRegressor(), param_grid, cv=knife, scoring='neg_mean_squared_error', return_train_score=True)
    grid_search.fit(X_train, y_train)
    XGB_pred = grid_search.predict(X_val)
    pred_training= grid_search.predict(X_train)
    feature_imp = grid_search.best_estimator_.feature_importances_
    return XGB_pred, pred_training, feature_imp


In [11]:
XG_pred, XG_train, feature_imp = XGBR(X_train, y_train, X_val)

In [251]:
all_result

[]

In [12]:
np.max(feature_imp)

0.5397962685946602

In [253]:
np.where(feature_imp == np.max(feature_imp))

(array([314]),)

In [220]:
# # Feature Column trigram
# lst = ['E','G','L','Y','T','H','R','A','C','D','P','I','F','N','K','S','V','M','W','Q']
# all_dct = {}
# key = []
# for i in lst:
#     for j in lst:
#         for k in lst:
#             st = i+j+k
#             all_dct[st] = []
# all_dct['temp'] = []

# for i, j in zip(all_dct.keys(),feature_imp):
#     all_dct[i] = j
# Importance_df = pd.DataFrame.from_dict([all_dct])

In [266]:
# # Feature Column bigram
# lst = ['E','G','L','Y','T','H','R','A','C','D','P','I','F','N','K','S','V','M','W','Q']
# all_dct = {}
# key = []
# for i in lst:
#     for j in lst:
#         st = i+j
#         all_dct[st] = []
# all_dct['temp'] = []

# s= 0
# for i, j in zip(all_dct.keys(),feature_imp):
# #     s+=j
# # print(s)
#     all_dct[i] = j
# Importance_df = pd.DataFrame.from_dict([all_dct])

In [11]:
# Feature Column quadrogram
lst = ['E','G','L','Y','T','H','R','A','C','D','P','I','F','N','K','S','V','M','W','Q']
all_dct = {}
key = []
for i in lst:
    for j in lst:
        for k in lst:
            for l in lst:
                st = i+j+k+l
                all_dct[st] = []
all_dct['temp'] = []

for i, j in zip(all_dct.keys(),feature_imp):
    all_dct[i] = j
Importance_df = pd.DataFrame.from_dict(all_dct,orient='index')

In [12]:
Importance_df

,0
EEEE,0.000000
EEEG,0.000000
EEEL,0.000000
EEEY,0.000000
EEET,0.000000
...,...
QQQV,0.000000
QQQM,0.000000
QQQW,0.000000
QQQQ,0.000000


In [13]:
Importance_df.to_excel("RF_feature_importance_quadrogram.xlsx")  

In [15]:
np.sum(Importance_df)

0    1.0
dtype: float64